In [1]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# Settings
dataset_name='citeseer'
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)
nodesize = features.shape[0]

# Some preprocessing
features_tmp=features.copy()
features = preprocess_features(features).A
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
# Setup Surrogate model
idx_train=np.array(np.where(train_mask==1)).tolist()[0]
idx_val=np.array(np.where(val_mask==1)).tolist()[0]
idx_unlabeled=np.array(np.where(test_mask==1)).tolist()[0]
surrogate = GCN(nfeat=features.shape[1], nclass=single_label.max().item()+1,
                nhid=256, dropout=0, with_relu=False, with_bias=False, device='cpu').to('cpu')
surrogate.fit(features, adj, single_label, idx_train, idx_val, patience=100)
# Setup Attack Model
model = Metattack(surrogate, nnodes=adj.shape[0], feature_shape=features.shape,
        attack_structure=True, attack_features=False, device='cpu', lambda_=0).to('cpu')
# Attack
model.attack(features, adj, single_label, idx_train, idx_unlabeled, n_perturbations=30, ll_constraint=False)
modified_adj = model.modified_adj
# print(adj)
# print("shiy")
# print(modified_adj)

/home/zsp/ljx/PTDNet/utils.py:221: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
Perturbing graph:   0%|          | 0/30 [00:00<?, ?it/s]

GCN loss on unlabled data: 1.6922634840011597
GCN acc on unlabled data: 0.67
attack loss: 1.6842432022094727


Perturbing graph:   3%|▎         | 1/30 [06:36<3:11:25, 396.05s/it]

GCN loss on unlabled data: 1.6943943500518799
GCN acc on unlabled data: 0.677
attack loss: 1.6869292259216309


Perturbing graph:   7%|▋         | 2/30 [13:23<3:07:49, 402.47s/it]

...

Perturbing graph:  93%|█████████▎| 28/30 [2:17:54<05:02, 151.26s/it]

GCN loss on unlabled data: 1.7178038358688354
GCN acc on unlabled data: 0.647
attack loss: 1.7110645771026611


Perturbing graph:  97%|█████████▋| 29/30 [2:20:15<02:28, 148.17s/it]

GCN loss on unlabled data: 1.7221990823745728
GCN acc on unlabled data: 0.673
attack loss: 1.7169256210327148


Perturbing graph: 100%|██████████| 30/30 [2:22:37<00:00, 285.27s/it]


In [2]:
modified_adj=sp.csr_array(modified_adj.int())

In [3]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        #nuclear = model.my_nuclear()
        nuclear = model.nuclear()
        loss = cross_loss + args.weight_decay*lossL2 + args.lambda1*lossl0 + args.lambda3*nuclear + args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))


/home/zsp/ljx/PTDNet/utils.py:221: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


edge_vol 4671.43
Epoch: 0001 train_loss= 1.79189 val_loss= 1.79109 train_acc= 0.39167 val_acc= 0.27000 best_val_acc_trail= 0.27000 test_acc= 0.26100
time  5.9573376178741455
edge_vol 4659.8486
Epoch: 0002 train_loss= 1.78970 val_loss= 1.79038 train_acc= 0.65833 val_acc= 0.33600 best_val_acc_trail= 0.33600 test_acc= 0.34200
time  7.130953788757324
edge_vol 4647.836
Epoch: 0003 train_loss= 1.78745 val_loss= 1.78965 train_acc= 0.83333 val_acc= 0.39600 best_val_acc_trail= 0.39600 test_acc= 0.40600
time  7.9679787158966064
edge_vol 4635.4077
Epoch: 0004 train_loss= 1.78523 val_loss= 1.78887 train_acc= 0.88333 val_acc= 0.43400 best_val_acc_trail= 0.43400 test_acc= 0.46100
time  8.696791887283325
edge_vol 4622.6357
Epoch: 0005 train_loss= 1.78293 val_loss= 1.78803 train_acc= 0.89167 val_acc= 0.47000 best_val_acc_trail= 0.47000 test_acc= 0.49900
time  9.55448055267334
edge_vol 4609.619
Epoch: 0006 train_loss= 1.78057 val_loss= 1.78713 train_acc= 0.89167 val_acc= 0.49800 best_val_acc_trail= 0.4

In [4]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        nuclear = model.my_nuclear()
        #nuclear = model.nuclear()
        loss = cross_loss + args.weight_decay*lossL2 + args.lambda1*lossl0 + args.lambda3*nuclear + args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))


edge_vol 4686.4883
Epoch: 0001 train_loss= 1.79187 val_loss= 1.79135 train_acc= 0.41667 val_acc= 0.18200 best_val_acc_trail= 0.18200 test_acc= 0.20100
time  3.7035741806030273
edge_vol 4682.6943
Epoch: 0002 train_loss= 1.78966 val_loss= 1.79062 train_acc= 0.64167 val_acc= 0.28000 best_val_acc_trail= 0.28000 test_acc= 0.29700
time  4.431981325149536
edge_vol 4676.705
Epoch: 0003 train_loss= 1.78746 val_loss= 1.78989 train_acc= 0.79167 val_acc= 0.35400 best_val_acc_trail= 0.35400 test_acc= 0.37700
time  5.1718645095825195
edge_vol 4669.4854
Epoch: 0004 train_loss= 1.78521 val_loss= 1.78912 train_acc= 0.87500 val_acc= 0.40400 best_val_acc_trail= 0.40400 test_acc= 0.44900
time  5.8780505657196045
edge_vol 4661.4463
Epoch: 0005 train_loss= 1.78294 val_loss= 1.78831 train_acc= 0.90000 val_acc= 0.45600 best_val_acc_trail= 0.45600 test_acc= 0.49500
time  6.589176893234253
edge_vol 4652.9023
Epoch: 0006 train_loss= 1.78060 val_loss= 1.78743 train_acc= 0.91667 val_acc= 0.49200 best_val_acc_trail

/home/zsp/ljx/PTDNet/models.py:14: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


edge_vol 9288.0
Epoch: 0317 train_loss= 0.49827 val_loss= 1.14308 train_acc= 0.98333 val_acc= 0.71800 best_val_acc_trail= 0.72000 test_acc= 0.70200
time  242.93161344528198
edge_vol 9288.0
Epoch: 0318 train_loss= 0.49624 val_loss= 1.14153 train_acc= 0.98333 val_acc= 0.71600 best_val_acc_trail= 0.72000 test_acc= 0.70200
time  243.67471385002136
edge_vol 9288.0
Epoch: 0319 train_loss= 0.49425 val_loss= 1.14002 train_acc= 0.98333 val_acc= 0.72000 best_val_acc_trail= 0.72000 test_acc= 0.70200
time  244.3078317642212
edge_vol 9288.0
Epoch: 0320 train_loss= 0.49323 val_loss= 1.13848 train_acc= 0.98333 val_acc= 0.72000 best_val_acc_trail= 0.72000 test_acc= 0.70200
time  245.13775634765625
edge_vol 9288.0
Epoch: 0321 train_loss= 0.49006 val_loss= 1.13696 train_acc= 0.98333 val_acc= 0.71800 best_val_acc_trail= 0.72000 test_acc= 0.70200
time  245.78100562095642
edge_vol 9288.0
Epoch: 0322 train_loss= 0.48896 val_loss= 1.13548 train_acc= 0.98333 val_acc= 0.71800 best_val_acc_trail= 0.72000 test_a

In [5]:
from config import args
import tensorflow as tf
import time
from utils import *
from models import GCN_dropedge
from metrics import *

# Settings
optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
args.dataset=dataset_name

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj

tuple_adj = sparse_to_tuple(adj.tocoo())
adj_tensor = tf.SparseTensor(*tuple_adj)

features = preprocess_features(features)

model = GCN_dropedge(input_dim=features.shape[1], output_dim=y_train.shape[1], adj=adj_tensor)


features_tensor = tf.convert_to_tensor(features,dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=tf.float32)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=tf.float32)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=tf.float32)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000


curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss + args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call((features_tensor), training=False)
    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        # Print results

    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(val_acc), "val_acc=", "{:.5f}".format(val_acc),
      "test_acc=", "{:.5f}".format(best_test_acc))

Epoch: 0001 train_loss= 1.79161 val_loss= 1.79104 train_acc= 0.25800 val_acc= 0.25800 test_acc= 0.26400
Epoch: 0002 train_loss= 1.78957 val_loss= 1.79033 train_acc= 0.35600 val_acc= 0.35600 test_acc= 0.37000
Epoch: 0003 train_loss= 1.78751 val_loss= 1.78959 train_acc= 0.42600 val_acc= 0.42600 test_acc= 0.45600
Epoch: 0004 train_loss= 1.78543 val_loss= 1.78881 train_acc= 0.50200 val_acc= 0.50200 test_acc= 0.53500
Epoch: 0005 train_loss= 1.78330 val_loss= 1.78798 train_acc= 0.55600 val_acc= 0.55600 test_acc= 0.57800
Epoch: 0006 train_loss= 1.78107 val_loss= 1.78707 train_acc= 0.59600 val_acc= 0.59600 test_acc= 0.61100
Epoch: 0007 train_loss= 1.77871 val_loss= 1.78609 train_acc= 0.62800 val_acc= 0.62800 test_acc= 0.63300
Epoch: 0008 train_loss= 1.77622 val_loss= 1.78501 train_acc= 0.65200 val_acc= 0.65200 test_acc= 0.65100
Epoch: 0009 train_loss= 1.77357 val_loss= 1.78386 train_acc= 0.66600 val_acc= 0.66600 test_acc= 0.65400
Epoch: 0010 train_loss= 1.77076 val_loss= 1.78262 train_acc= 0.6